In [1]:
import sys
import os
import numpy as np
PLNX_LIBS_PATH = "/Users/alex/Dev_projects/MyOwnRepo/poloniex_api/src"
RT_LIBS_PATH = "/Users/alex/Dev_projects/MyOwnRepo/rt_libs/src"
BA_LIBS_PATH = "/Users/alex/Dev_projects/MyOwnRepo/basic_application/src"
sys.path.append(PLNX_LIBS_PATH)
sys.path.append(RT_LIBS_PATH)
sys.path.append(BA_LIBS_PATH)

In [2]:
from poloniex import PublicAPI, PublicAPIError
import time

import numpy as np
import pandas as pd

import pickle

from data_providers import ClickHouseConnector
from data_providers import DbDataProviderUniversal

In [3]:
%matplotlib notebook
db_connect_params = {
    "host" : "217.197.116.177",
    "port" : 59000,
    "user" : "alex",
    "password" : "Xrxcmr758",
    "database" : "rt"
}

In [4]:
poloniex = PublicAPI()

In [19]:
currencyPair = "BTC_XRP"
start = "2022-07-10 00:00:00"
end = "2022-07-10 01:00:00"

data = poloniex.get_trade_history(currencyPair, start, end)

In [21]:
data

[{'globalTradeID': 708267503,
  'tradeID': 31901514,
  'date': '2022-07-10 00:59:29',
  'type': 'buy',
  'rate': '0.00001599',
  'amount': '45.84766255',
  'total': '0.00073310',
  'orderNumber': 446994851823},
 {'globalTradeID': 708267460,
  'tradeID': 31901513,
  'date': '2022-07-10 00:59:20',
  'type': 'buy',
  'rate': '0.00001599',
  'amount': '566.56512788',
  'total': '0.00905937',
  'orderNumber': 446994824850},
 {'globalTradeID': 708267271,
  'tradeID': 31901512,
  'date': '2022-07-10 00:58:21',
  'type': 'buy',
  'rate': '0.00001596',
  'amount': '32.99031207',
  'total': '0.00052652',
  'orderNumber': 446994747927},
 {'globalTradeID': 708267153,
  'tradeID': 31901511,
  'date': '2022-07-10 00:57:49',
  'type': 'sell',
  'rate': '0.00001597',
  'amount': '46.72093596',
  'total': '0.00074613',
  'orderNumber': 446994718956},
 {'globalTradeID': 708266829,
  'tradeID': 31901510,
  'date': '2022-07-10 00:56:23',
  'type': 'buy',
  'rate': '0.00001598',
  'amount': '47.48639227',


In [ ]:
start
end
period
pairs
step=60



with ClickHouseConnector(db_connect_params) as connector:
    dp = DbDataProviderUniversal(connector, q)

    
    data = dp.get()


In [8]:
with ClickHouseConnector(db_connect_params) as connector:
    connector.cursor.execute(q)
    raw_data = connector.cursor.fetchall()
    data_length = connector.cursor.rowcount

Cursor created, database connection established
Cursor closed


OperationalError: Code: 62.
DB::Exception: Syntax error: failed at position 21 ('%') (line 4, col 14): %(period)s) AS p_period,
    toUInt32(%(start)s) AS ts_min,
    toUInt32(%(end)s+p_period) AS ts_max,
    toUInt32(%(step)s) AS step,
    %(pair)s AS p_pair


S. Expected one of: ALL, DISTINCT, list of expressions, list of elements, expression with optional alias, element of expression with optional alias, lambda expression, token, OpeningRoundBracket, identifier, Arrow, expression with ternary operator, logical-OR expression, list, delimited by operator of variable arity, logical-AND expression, logical-NOT expression, expression with prefix unary operator, NOT, nullity checking, comparison expression, list, delimited by binary operators, BETWEEN expression, string concatenation expression, additive expression, INTERVAL operator expression, INTERVAL, TIMESTAMP operator expression, TIMESTAMP, DATE operator expression, DATE, multiplicative expression, unary expression, CAST expression,

##  Пост обработка

In [304]:
# функция расчета highest_bid,lowest_ask
def get_rates(asks, bids):
    lowest_ask = np.min(np.array(list(map(float, asks.keys()))))
    highest_bid = np.max(np.array(list(map(float, bids.keys()))))
    return lowest_ask, highest_bid
# ----------------------------------------

# функция расчета маски
LEVELS = [0.005, 0.01, 0.02, 0.03, 0.05]
def get_volumes(asks, bids, lowest_ask, highest_bid):
    keys_asks = np.array(list(map(float, asks.keys())))
    vols_asks = np.array(list(map(float, asks.values())))
    keys_bids = np.array(list(map(float, bids.keys())))
    vols_bids = np.array(list(map(float, bids.values())))
    
    asks_sum, bids_sum = [], []
    for coef in LEVELS:
        mask_ask = keys_asks < lowest_ask * (1 + coef)
        asks_sum.append(sum(vols_asks[mask_ask]))

        mask_bid = keys_bids > highest_bid * (1 - coef)
        bids_sum.append(sum(vols_bids[mask_bid]))
    
    return np.array(asks_sum), np.array(bids_sum)

In [306]:
row_current = raw_data[0]
asks = row_current[2]
bids = row_current[3]

lowest_ask, highest_bid = get_rates(asks, bids)

asks_vol, bids_vol = get_volumes(asks, bids, lowest_ask, highest_bid)

In [311]:
row_previous = raw_data[1]
asks = row_previous[2]
bids = row_previous[3]

asks_vol_prev, bids_vol_pev = get_volumes(asks, bids, lowest_ask, highest_bid)

In [315]:
asks_vol - asks_vol_prev

array([-0.90830162, -0.90637021, -0.90637021, -0.90637021, -0.90637021])

# Арпоксимация orderbook

In [394]:
pair = "BTC_XMR"
ob_1 = api.get_orderbook(pair, 100)
ts = int(time.time())

In [395]:
asks = dict(ob_1["asks"])
bids = dict(ob_1["bids"])

isFrozen = bool(int(ob_1["isFrozen"]))
postOnly = bool(int(ob_1["postOnly"]))

data = []
record = [ts, pair, asks, bids, isFrozen, postOnly]
data.append(record)

In [420]:
LEVELS = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.9, 1.1, 1.3, 1.5, 1.75, 2.0, 2.25, 2.5, 2.75, 3.0, 100]

LEVELS = [0, 0.2, 0.4, 0.6, 0.8, 1, 1.25, 1.5, 1.75, 2, 2.25, 2.5, 2.75, 3, 3.5, 4, 4.5, 5, 5.5, 6, 6.5, 7, 7.5, 100]

In [424]:
def resample_ob(asks, bids):
    
    TEMPL = "{:.8f}"

    keys_asks = np.array(list(map(float, asks.keys())))
    vols_asks = np.array(list(map(float, asks.values())))
    keys_bids = np.array(list(map(float, bids.keys())))
    vols_bids = np.array(list(map(float, bids.values())))

    lowest_ask = min(keys_asks)
    highest_bid = max(keys_bids)

    asks_resampled = dict()
    key = TEMPL.format(lowest_ask)
    asks_resampled[key] = asks[key]

    bids_resampled = dict()
    key = TEMPL.format(highest_bid)
    bids_resampled[key] = bids[key]

    for idx in np.arange(1, len(LEVELS)):
        mask_ask = (keys_asks > lowest_ask * (1 + LEVELS[idx-1]/100.)) & (keys_asks <= lowest_ask * (1 + LEVELS[idx]/100.))
        value = np.round(sum(vols_asks[mask_ask]), 8)
        key = TEMPL.format(lowest_ask * (1 + LEVELS[idx]/100.))
        asks_resampled[key] = value

        mask_bid = (keys_bids < highest_bid * (1 - LEVELS[idx-1]/100.)) & (keys_bids >= highest_bid * (1 - LEVELS[idx]/100.))
        value = np.round(sum(vols_bids[mask_bid]),8 )
        key = TEMPL.format(highest_bid * (1 - LEVELS[idx]/100.))
        bids_resampled[key] = value
        
    return asks_resampled, bids_resampled


asks_resampled, bids_resampled = resample_ob(asks, bids)
    
    

    
    
    

In [425]:
asks_resampled

{'0.00616191': 0.37541865,
 '0.00617423': 15.95311894,
 '0.00618656': 5.0,
 '0.00619888': 59.7,
 '0.00621121': 0.08529045,
 '0.00622353': 6.92699843,
 '0.00623893': 0.55705701,
 '0.00625434': 28.89005107,
 '0.00626974': 1.14874215,
 '0.00628515': 0.4,
 '0.00630055': 53.32383531,
 '0.00631596': 6.30613541,
 '0.00633136': 0,
 '0.00634677': 0.98149297,
 '0.00637758': 53.22707403,
 '0.00640839': 65.04185393,
 '0.00643920': 0.84741609,
 '0.00647001': 60.35126729,
 '0.00650082': 62.38967033,
 '0.00653162': 5.71407428,
 '0.00656243': 53.96412118,
 '0.00659324': 8.51375563,
 '0.00662405': 53.81567377,
 '0.01232382': 175.25381259}

In [426]:
bids_resampled

{'0.00615560': 0.02162258,
 '0.00614329': 0.04878049,
 '0.00613098': 3.63287452,
 '0.00611867': 77.56280331,
 '0.00610636': 0.16357135,
 '0.00609404': 5.82440337,
 '0.00607865': 31.42230216,
 '0.00606327': 14.582437,
 '0.00604788': 0.09251823,
 '0.00603249': 5.27938537,
 '0.00601710': 0.61451573,
 '0.00600171': 0.06351775,
 '0.00598632': 55.64323124,
 '0.00597093': 11.35828994,
 '0.00594015': 53.97395532,
 '0.00590938': 6.04083334,
 '0.00587860': 55.26618697,
 '0.00584782': 59.1282941,
 '0.00581704': 0.90827999,
 '0.00578626': 59.98792606,
 '0.00575549': 0.66564304,
 '0.00572471': 60.54629841,
 '0.00569393': 56.64729748,
 '0.00000000': 6.1963242}

In [287]:
for idx in np.arange(1, len(raw_data), 1):

    last_row = raw_data[idx-1]
    curr_row = raw_data[idx]
    
    asks = last_row[2]
    bids = last_row[3]
    
    lowest_ask, highest_bid = get_rates(asks, bids)
    
    ask_vol, 
    
    

In [6]:
api = PublicAPI()

In [175]:
pair = "USDT_BTC"
ob_1 = api.get_orderbook(pair, 30)
#time.sleep(10)

In [56]:
asks_1 = dict(ob_1["asks"])
bids_1 = dict(ob_1["bids"])

asks_2 = dict(ob_2["asks"])
bids_2 = dict(ob_2["bids"])

In [174]:
ob_2

{'asks': [['19433.70000000', 9.9e-05],
  ['19443.70565382', 0.00176794],
  ['19443.70566383', 0.17],
  ['19448.65228888', 0.15],
  ['19448.91647128', 0.00332021],
  ['19457.47191502', 0.5],
  ['19460.62054852', 0.00231746],
  ['19465.49463482', 0.00239519],
  ['19466.90840497', 0.23748013],
  ['19467.00000000', 0.000504],
  ['19472.60580486', 0.00223101],
  ['19477.90856022', 0.00210157],
  ['19487.44846671', 0.85323017],
  ['19492.20000000', 0.188],
  ['19495.35825471', 0.0149876],
  ['19506.80000000', 0.39],
  ['19508.15435397', 0.00308486],
  ['19522.45872000', 0.0024319],
  ['19525.20000000', 0.71],
  ['19529.27356505', 0.49482612],
  ['19540.31550000', 3.95],
  ['19544.05069752', 0.37399885],
  ['19548.00000000', 0.00298748],
  ['19560.98099837', 0.00103124],
  ['19593.22214360', 0.00013858],
  ['19605.69961398', 0.00306951],
  ['19615.16876207', 0.04922],
  ['19629.56307120', 0.0009],
  ['19635.45127267', 0.03006042],
  ['19638.02556690', 0.00013827],
  ['19639.45000000', 0.907],

ts
pair
orderbook


## Highest_bid

In [57]:
LEVELS = [0.005, 0.01, 0.02, 0.03, 0.05]

lowest_ask = np.min(np.array(list(map(float, asks_1.keys()))))
highest_bid = np.max(np.array(list(map(float, bids_1.keys()))))

keys_asks_1 = np.array(list(map(float, asks_1.keys())))
vols_asks_1 = np.array(list(map(float, asks_1.values())))
keys_bids_1 = np.array(list(map(float, bids_1.keys())))
vols_bids_1 = np.array(list(map(float, bids_1.values())))


keys_asks_2 = np.array(list(map(float, asks_2.keys())))
vols_asks_2 = np.array(list(map(float, asks_2.values())))
keys_bids_2 = np.array(list(map(float, bids_2.keys())))
vols_bids_2 = np.array(list(map(float, bids_2.values())))


In [109]:
ob_1 = api.get_orderbook(pair, 100)

# функция расчета highest_bid,lowest_ask
def get_rates(asks, bids):

    lowest_ask = np.min(np.array(list(map(float, asks.keys()))))
    highest_bid = np.max(np.array(list(map(float, bids.keys()))))
    return lowest_ask, highest_bid

# ----------------------------------------
asks = dict(ob_1["asks"])
bids = dict(ob_1["bids"])

lowest_ask, highest_bid = get_rates(asks, bids)

print(lowest_ask)
print(highest_bid)




def get_rate(orderbook, side='ask'):
    if side == "ask":
        # lowest_ask
        value = np.min(np.array(list(map(float, asks.keys()))))
    else:
        # highest_bid
        value = np.max(np.array(list(map(float, bids.keys()))))
    return value

# ----------------------------------------
asks = dict(ob_1["asks"])
bids = dict(ob_1["bids"])

lowest_ask = get_rate(asks, side='ask')
highest_bid = get_rate(bids, side='ask')

print(lowest_ask)
print(highest_bid)

43.42586105
40.42930147


In [113]:
# функция расчета маски
def get_mask(orderbook, rate, threshold, side='ask'):
    keys = np.array(list(map(float, orderbook.keys())))
    vols = np.array(list(map(float, orderbook.values())))
    
    if side == "ask":
        mask = keys < rate * (1 + threshold)
    else:
        mask = keys > rate * (1 - threshold)
    return mask

# ----------------------------------------

ob = api.get_orderbook(pair, 100)
threshold = 0.01

asks = dict(ob["asks"])
lowest_ask = get_rate(asks, side='ask')
mask_ask = get_mask(asks, lowest_ask, threshold, side='ask')

bids = dict(ob["bids"])
highest_bid = get_rate(bids, side='bid')
mask_bid = get_mask(bids, highest_bid, threshold, side='bid')

print(mask_ask, "\n\n", mask_bid)

[ True  True False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False] 

 [ True False False False False False False False False False False False
 False False False False False False False False False False]


In [114]:
pair = "USDC_DASH"
ob_1 = api.get_orderbook(pair, 100)

In [ ]:
ob_2 = api.get_orderbook(pair, 100)

In [115]:
# Функция для расчета объемов по маске
LEVELS = [0.005, 0.01, 0.02, 0.03, 0.05]

def get_volume(orderbook, mask):
    keys_asks = np.array(list(map(float, asks_1.keys())))
    vols_asks = np.array(list(map(float, asks_1.values())))

    volume = sum(orderbook[mask])
    return volume

# ============================================
threshold = 0.01

asks = dict(ob_1["asks"])
lowest_ask = get_rate(asks, side='ask')
mask_ask = get_mask(asks, lowest_ask, threshold, side='ask')
volume_1 = get_volume(asks, mask_ask)

asks_2 = dict(ob_2["asks"])
mask_ask_2 = get_mask(asks_2, lowest_ask, threshold, side='ask')
volume_2 = get_volume(asks_2, mask_ask_2)



TypeError: unhashable type: 'numpy.ndarray'

In [176]:
pair = "USDT_BTC"
ob_1 = api.get_orderbook(pair, 100)

In [200]:
ob_2 = api.get_orderbook(pair, 1001)

In [201]:
LEVELS = [0.01, 0.02, 0.03, 0.05, 0.08]

# функция расчета highest_bid,lowest_ask
def get_rates(orderbook):
    asks = dict(orderbook["asks"])
    bids = dict(orderbook["bids"])
    
    lowest_ask = np.min(np.array(list(map(float, asks.keys()))))
    highest_bid = np.max(np.array(list(map(float, bids.keys()))))
    return lowest_ask, highest_bid


def get_volumes(orderbook, lowest_ask, highest_bid):
    asks = dict(orderbook["asks"])
    bids = dict(orderbook["bids"])
    
    keys_asks = np.array(list(map(float, asks.keys())))
    vols_asks = np.array(list(map(float, asks.values())))
    keys_bids = np.array(list(map(float, bids.keys())))
    vols_bids = np.array(list(map(float, bids.values())))

    asks_sum, bids_sum = [], []
    
    for coef in LEVELS:
        mask_ask = keys_asks < lowest_ask * (1 + coef)
        asks_sum.append(sum(vols_asks[mask_ask]))

        mask_bid = keys_bids > highest_bid * (1 - coef)
        bids_sum.append(sum(vols_bids[mask_bid]))
    
    return np.array(asks_sum), np.array(bids_sum)


lowest_ask, highest_bid = get_rates(ob_1)
vol_asks, vol_bids = get_volumes(ob_1, lowest_ask, highest_bid)





In [202]:
vol_asks_last, vol_bids_last = get_volumes(ob_2, lowest_ask, highest_bid)

In [203]:
print(vol_asks)
print(vol_asks_last)
print(vol_asks_last - vol_asks)

[11.81066286 11.86399095 11.86399095 11.86399095 11.86399095]
[ 9.07912039  9.21467345 11.17121359 11.64742202 11.64742202]
[-2.73154247 -2.6493175  -0.69277736 -0.21656893 -0.21656893]


In [204]:
print(vol_bids)
print(vol_bids_last)
print(vol_bids_last - vol_bids)

[12.56790432 12.56790432 12.56790432 12.56790432 12.56790432]
[ 5.04439496 14.63231595 20.74138815 20.74138815 20.74138815]
[-7.52350936  2.06441163  8.17348383  8.17348383  8.17348383]


array([-1.53487451, -1.58806402, -1.58806402, -1.58806402, -1.58806402])

array([-0.05692265, -0.00377968, -0.00377968, -0.00377968, -0.00377968])

In [117]:
asks

{'43.66600000': 0.054605,
 '45.66600000': 0.05748,
 '47.66600000': 0.05505,
 '49.66600000': 0.057945,
 '50.00000000': 3.8e-06,
 '67.77000000': 0.16642611,
 '75.08148519': 0.02,
 '79.08305123': 0.05178608,
 '80.00000000': 0.1336616,
 '80.66299221': 0.05235888,
 '80.83699378': 0.02,
 '82.36767366': 0.05288247,
 '84.19872965': 0.05341129,
 '85.10393666': 0.02,
 '86.15863256': 0.05394541,
 '88.25058966': 0.05448486,
 '89.09543172': 0.02,
 '90.47847510': 0.05502971,
 '92.84678616': 0.05558001,
 '95.36061672': 0.05613581,
 '98.02564355': 0.05669716,
 '100.84812242': 0.05726414,
 '106.99338486': 0.05783678,
 '123.45000000': 5.07983308,
 '125.00000000': 0.017616,
 '141.47143049': 0.29360566,
 '180.10000000': 0.1458097,
 '186.00000000': 0.0158656,
 '188.00000000': 5.65243853,
 '198.00000000': 1.1,
 '200.00000000': 6.01934952,
 '206.00000000': 0.47964101,
 '207.00000000': 0.35973076,
 '210.00000000': 11.36832018,
 '214.18730630': 0.0151,
 '220.00000000': 1.42592169,
 '220.69481332': 0.00533233,


In [64]:
vols_asks_1[mask_ask]

array([20., 20.])

In [62]:
asks = []
bids = []
for coef in LEVELS:
    mask_ask = keys_asks_1 < lowest_ask * (1 + coef)
    print(len(mask_ask))
    print(len(vols_asks_1))
    
    volume_1 = sum(vols_asks_1[mask_ask])
    volume_2 = sum(vols_asks_2[mask_ask])
    
    asks.append(volume_2-volume_1)

    mask_bid = keys_bids > highest_bid * (1 - coef)
    volume_1 = sum(vols_bids_1[mask_ask])
    volume_2 = sum(vols_bids_2[mask_ask])
    
    bids.append(volume_2-volume_1)

100
100


IndexError: boolean index did not match indexed array along dimension 0; dimension is 23 but corresponding boolean dimension is 100